In [9]:
import pandas as pd
import torch
from tqdm import tqdm
from sacrebleu.metrics import BLEU
from bert_score import score
from googletrans import Translator
import time
import numpy as np


In [ ]:
# Config
SRC = 'PDF1'
FROM_ID = 501
TO_ID = 625
INPUT_FILE = f"../output/alignment/{SRC}_{FROM_ID}_{TO_ID}.csv" 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")
    
    # Lấy mẫu ngẫu nhiên
    sample = df.sample(n=min(sample_size, len(df)), random_state=0).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file 
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0
    
    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset
        
        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5: 
            continue
            
        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text
            
            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated) 
            hypotheses.append(tgt)
            
            success_count += 1
            time.sleep(0.5)
            
        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue
            
    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])
    
    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)
    
    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")

In [12]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
SAMPLE_SIZE = int(df.shape[0] * 0.1)  # Số lượng câu lấy mẫu để đánh giá 
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 149 cặp câu để đánh giá...


100%|██████████| 149/149 [04:48<00:00,  1.94s/it]


Đã dịch thành công 149 câu.
BLEU Score: 45.98
BERTScore (F1): 0.9033
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: Bằng cách sử dụng công cụ đường dẫn nhân quả của họ, chúng tôi đã có thể xác định các đường dẫn bị ảnh hưởng khác biệt nhất, không chỉ các protein riêng lẻ từ bản đồ nhiệt đó.
Google Trans: Bằng cách sử dụng công cụ lộ trình nhân quả của họ, chúng tôi có thể xác định những con đường bị ảnh hưởng nhiều nhất, không chỉ các protein riêng lẻ trong bản đồ nhiệt.
Cặp 2:
Dataset Tgt: và nói rằng, tốt, chúng tôi có hai tin tốt cho bạn.
Google Trans: Và nói rằng, chúng tôi có hai tin tốt cho bạn.
Cặp 3:
Dataset Tgt: Liều lượng cho một người 18 tuổi.
Google Trans: Liều dùng cho trẻ 18 tuổi.
